In [14]:
import re
import requests
from bs4 import BeautifulSoup
from reportlab.lib.pagesizes import A4
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import SimpleDocTemplate, Paragraph

def scraping_do_texto(link):
    # Função que realiza o scraping do texto a partir de um link fornecido.
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'
    }
    requisicao = requests.get(link, headers=headers)
    
    if requisicao.status_code == 200:
        site = BeautifulSoup(requisicao.text, 'html.parser')
        paragrafos = site.find_all('p')
        texto = ' '.join([paragrafo.get_text() for paragrafo in paragrafos])
        return texto
    else:
        print(f"Erro ao acessar o site: {requisicao.status_code}")
        return None

def estruturar_texto(texto):
    # Função que estrutura o texto em capítulos e parágrafos.
    # Separar capítulos
    capitulos = re.split(r'\*\*(.*?)\*\*', texto)
    estrutura = []
    for i in range(0, len(capitulos), 2):
        pre_capitulo = capitulos[i].strip()
        if i + 1 < len(capitulos):
            titulo_capitulo = capitulos[i + 1].strip()
            if titulo_capitulo:
                estrutura.append(f"\nCapítulo: {titulo_capitulo}")
        else:
            titulo_capitulo = ""
        # Separar parágrafos dentro do capítulo
        paragrafos = re.split(r'###', pre_capitulo)
        for paragrafo in paragrafos:
            paragrafo = paragrafo.strip()
            if paragrafo:
                estrutura.append(f"\nParágrafo:\n{paragrafo}")
    return '\n'.join(estrutura)

def salvar_como_pdf(texto, nome_arquivo):
    # Função que salva o texto estruturado em um arquivo PDF.
    # Configurar o documento PDF
    pdf = SimpleDocTemplate(nome_arquivo, pagesize=A4)
    estilos = getSampleStyleSheet()
    elementos = []

    # Adicionar cada linha do texto como um parágrafo
    for linha in texto.split('\n'):
        if linha.strip():  # Ignorar linhas vazias
            paragrafo = Paragraph(linha, estilos['BodyText'])
            elementos.append(paragrafo)

    # Construir o PDF
    pdf.build(elementos)

def main():
    link = input('Digite o link: ')
    texto = scraping_do_texto(link)
    if texto:
        texto_estruturado = estruturar_texto(texto)
        salvar_como_pdf(texto_estruturado, 'texto_estruturado.pdf')
        print('PDF gerado com sucesso.')
    else:
        print('Não foi possível obter o texto do link fornecido.')

if __name__ == '__main__':
    main()

Digite o link:  https://www.blog.schooladvisor.com.br/guia-escolas-belo-horizonte


PDF gerado com sucesso.
